# PATH SETUP 

In [19]:
from pathlib import Path
import sys

try:
    # Running as .py script
    FILE_PATH = Path(__file__).resolve()
    PROJECT_ROOT = FILE_PATH.parents[2]  # crypto-volatility-ml
except NameError:
    # Running inside Jupyter (cwd = crypto-volatility-ml/notebooks)
    PROJECT_ROOT = Path.cwd().resolve().parents[0]

DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REPORT_DIR = PROJECT_ROOT / "reports"
FIG_DIR = REPORT_DIR / "figures"

# Create folders if missing
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("FIG_DIR:", FIG_DIR)


PROJECT_ROOT: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml
PROCESSED_DIR: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\data\processed
FIG_DIR: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures


## # AUTO ARIMA MODEL — FULL PIPELINE WITH DETAILED LOGGING (p,d,q)

In [26]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from pmdarima import auto_arima

# Paths
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
FIG_DIR = PROJECT_ROOT / "reports" / "figures"
DATA_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Load cleaned dataset
df = pd.read_csv(PROCESSED_DIR / "crypto_features.csv", parse_dates=["Date"])
df = df.sort_values(["Name","Date"]).reset_index(drop=True)

# ------------------------------------------------------------
# Plot savers
# ------------------------------------------------------------
def save_forecast_plot(dates, actual, predicted, coin, model_name):
    plt.figure(figsize=(11,5))
    plt.plot(dates, actual, label="Actual", linewidth=2)
    plt.plot(dates, predicted, label="Predicted", linewidth=2)
    plt.title(f"{model_name} Forecast vs Actual — {coin}")
    plt.xlabel("Date"); plt.ylabel("Price")
    plt.legend()
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_forecast.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path

def save_residual_plot(dates, residuals, coin, model_name):
    plt.figure(figsize=(11,5))
    plt.plot(dates, residuals, color="purple")
    plt.axhline(0, color="black", linestyle="--")
    plt.title(f"{model_name} Residuals — {coin}")
    plt.xlabel("Date"); plt.ylabel("Residual")
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_residuals.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path

def save_predictions_csv(dates, actual, predicted, coin, model_name):
    out = pd.DataFrame({
        "Date": dates,
        "y_true": actual,
        "y_pred": predicted
    })
    out_path = PROCESSED_DIR / f"predictions_{model_name.lower()}_{coin}.csv"
    out.to_csv(out_path, index=False)
    return out_path

# ------------------------------------------------------------
# RUN AUTO ARIMA FOR EACH COIN
# ------------------------------------------------------------
results_auto_arima = []

print("\n================= AUTO ARIMA FORECASTING START =================")

for coin in df["Name"].unique():
    sub = df[df["Name"] == coin][["Date", "Close"]].dropna()

    if len(sub) < 200:
        print(f" Skipping {coin}: Not enough rows\n")
        continue

    print("\n----------------------------------------------------------")
    print(f" Now Processing Coin: {coin.upper()}")
    print("----------------------------------------------------------")

    # 80/20 Time-Based Split
    split = int(len(sub) * 0.8)
    train = sub.iloc[:split]
    test  = sub.iloc[split:]

    y_train = train["Close"].values
    y_test  = test["Close"].values
    train_dates = train["Date"].values
    test_dates  = test["Date"].values

    # INTERVAL PRINTING
    print(f"• Train interval: {train['Date'].iloc[0].date()} → {train['Date'].iloc[-1].date()} ({len(train)} rows)")
    print(f"• Test interval : {test['Date'].iloc[0].date()} → {test['Date'].iloc[-1].date()} ({len(test)} rows)")

    # -------------------------
    # FIT AUTO ARIMA
    # -------------------------
    print(f" Fitting Auto ARIMA for {coin}...")

    model = auto_arima(
        y_train,
        seasonal=False,
        stepwise=True,
        max_p=5, max_q=5,
        max_order=10,
        suppress_warnings=True,
        error_action="ignore",
        trace=False
    )

    # Print p,d,q parameters
    order = model.order              # (p,d,q)
    seasonal_order = model.seasonal_order   # (P,D,Q,m)

    p, d, q = order
    P, D, Q, m = seasonal_order

    print(f" Selected (p,d,q) = ({p}, {d}, {q})")
    print(f" Seasonal (P,D,Q,m) = ({P}, {D}, {Q}, {m})")

    # -------------------------
    # TRAIN METRICS (in-sample)
    # -------------------------
    train_preds = model.predict_in_sample()

    train_mae = mean_absolute_error(y_train, train_preds)
    train_rmse = root_mean_squared_error(y_train, train_preds)

    print(f" Train MAE  = {train_mae:.3f}")
    print(f" Train RMSE = {train_rmse:.3f}")

    # -------------------------
    # TEST FORECAST
    # -------------------------
    print(f" Forecasting next {len(test)} points...")
    preds = model.predict(n_periods=len(test))

    # TEST METRICS
    test_mae  = mean_absolute_error(y_test, preds)
    test_rmse = root_mean_squared_error(y_test, preds)

    print(f" Test MAE   = {test_mae:.3f}")
    print(f" Test RMSE  = {test_rmse:.3f}")

    # -------------------------
    # SAVE PLOTS + CSV
    # -------------------------
    forecast_plot = save_forecast_plot(test_dates, y_test, preds, coin, "AutoARIMA")
    residual_plot = save_residual_plot(test_dates, y_test - preds, coin, "AutoARIMA")
    pred_csv      = save_predictions_csv(test_dates, y_test, preds, coin, "AutoARIMA")

    # -------------------------
    # COLLECT RESULTS
    # -------------------------
    results_auto_arima.append({
        "Coin": coin,
        "Model": "AutoARIMA",
        "p": p,
        "d": d,
        "q": q,
        "Train_MAE": train_mae,
        "Train_RMSE": train_rmse,
        "Test_MAE": test_mae,
        "Test_RMSE": test_rmse,
        "ForecastPlot": str(forecast_plot),
        "ResidualPlot": str(residual_plot),
        "PredCSV": str(pred_csv)
    })

# Convert results to nice table
print("\n================= AUTO ARIMA FINISHED =================\n")
auto_arima_df = pd.DataFrame(results_auto_arima).sort_values("Test_RMSE")
auto_arima_df



================= AUTO ARIMA FORECASTING START =================

----------------------------------------------------------
 Now Processing Coin: BITCOIN
----------------------------------------------------------
• Train interval: 2013-04-29 → 2019-11-15 (2392 rows)
• Test interval : 2019-11-16 → 2021-07-06 (599 rows)
 Fitting Auto ARIMA for Bitcoin...
 Selected (p,d,q) = (1, 1, 1)
 Seasonal (P,D,Q,m) = (0, 0, 0, 0)
 Train MAE  = 96.349
 Train RMSE = 248.732
 Forecasting next 599 points...
 Test MAE   = 13596.441
 Test RMSE  = 21812.776

----------------------------------------------------------
 Now Processing Coin: ETHEREUM
----------------------------------------------------------
• Train interval: 2015-08-08 → 2020-04-30 (1728 rows)
• Test interval : 2020-05-01 → 2021-07-06 (432 rows)
 Fitting Auto ARIMA for Ethereum...
 Selected (p,d,q) = (0, 1, 0)
 Seasonal (P,D,Q,m) = (0, 0, 0, 0)
 Train MAE  = 8.357
 Train RMSE = 19.515
 Forecasting next 432 points...
 Test MAE   = 901.868
 T

,Coin,Model,p,d,q,Train_MAE,Train_RMSE,Test_MAE,Test_RMSE,ForecastPlot,ResidualPlot,PredCSV
2,Litecoin,AutoARIMA,2,1,2,1.750229,4.836994,52.828273,85.415746,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
1,Ethereum,AutoARIMA,0,1,0,8.356520,19.514984,901.868197,1319.211703,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
0,Bitcoin,AutoARIMA,1,1,1,96.349074,248.732049,13596.441298,21812.776458,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...


## PROPHET MODEL

In [27]:
import warnings
warnings.filterwarnings("ignore")

from prophet import Prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# ------------------------------------------------------------
# PROPHET Plot savers (same pattern as ARIMA)
# ------------------------------------------------------------
def save_prophet_forecast_plot(dates, actual, predicted, coin, model_name="Prophet"):
    plt.figure(figsize=(11,5))
    plt.plot(dates, actual, label="Actual", linewidth=2)
    plt.plot(dates, predicted, label="Predicted", linewidth=2)
    plt.title(f"{model_name} Forecast vs Actual — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.tight_layout()

    out_path = FIG_DIR / f"{model_name}_{coin}_forecast.png"
    plt.savefig(str(out_path), dpi=240)
    plt.close()
    print(f" Saved forecast plot: {out_path}")
    return out_path


def save_prophet_residual_plot(dates, residuals, coin, model_name="Prophet"):
    plt.figure(figsize=(11,5))
    plt.plot(dates, residuals, color="purple")
    plt.axhline(0, color="black", linestyle="--")
    plt.title(f"{model_name} Residual Plot — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Residual")
    plt.tight_layout()

    out_path = FIG_DIR / f"{model_name}_{coin}_residuals.png"
    plt.savefig(str(out_path), dpi=240)
    plt.close()
    print(f" Saved residual plot: {out_path}")
    return out_path


def save_prophet_predictions_csv(dates, y_true, y_pred, coin, model_name="Prophet"):
    out = pd.DataFrame({
        "Date": pd.to_datetime(dates),
        "y_true": y_true,
        "y_pred": y_pred
    })
    out_path = PROCESSED_DIR / f"predictions_{model_name.lower()}_{coin}.csv"
    out.to_csv(out_path, index=False)
    return out_path


# ------------------------------------------------------------
# RUN PROPHET FOR EACH COIN
# ------------------------------------------------------------
results_prophet = []

print("\n================= PROPHET FORECASTING START =================")

for coin in df["Name"].unique():
    sub = df[df["Name"] == coin][["Date", "Close"]].dropna()

    if len(sub) < 300:
        print(f" Skipping {coin}: Not enough data (min 300 rows required)\n")
        continue

    print("\n----------------------------------------------------------")
    print(f" Now Processing Coin: {coin.upper()}")
    print("----------------------------------------------------------")

    split = int(len(sub) * 0.8)
    train = sub.iloc[:split].rename(columns={"Date":"ds", "Close":"y"})
    test  = sub.iloc[split:].rename(columns={"Date":"ds", "Close":"y"})

    # Interval logging
    print(f"• Train interval: {train['ds'].iloc[0].date()} → {train['ds'].iloc[-1].date()} ({len(train)} rows)")
    print(f"• Test interval : {test['ds'].iloc[0].date()} → {test['ds'].iloc[-1].date()} ({len(test)} rows)")

    # -------------------------
    # FIT PROPHET
    # -------------------------
    print(f" Fitting Prophet for {coin}...")

    model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        changepoint_prior_scale=0.5
    )

    model.fit(train)

    # -------------------------
    # TRAIN METRICS
    # -------------------------
    train_forecast = model.predict(train)
    train_pred = train_forecast["yhat"].values

    train_mae  = mean_absolute_error(train["y"], train_pred)
    train_rmse = root_mean_squared_error(train["y"], train_pred)

    print(f" Train MAE  = {train_mae:.3f}")
    print(f" Train RMSE = {train_rmse:.3f}")

    # -------------------------
    # TEST FORECAST
    # -------------------------
    future = model.make_future_dataframe(periods=len(test))
    forecast = model.predict(future)

    preds = forecast["yhat"].iloc[-len(test):].values
    y_true = test["y"].values
    dates  = test["ds"].values

    # TEST METRICS
    test_mae  = mean_absolute_error(y_true, preds)
    test_rmse = root_mean_squared_error(y_true, preds)

    print(f" Test MAE   = {test_mae:.3f}")
    print(f" Test RMSE  = {test_rmse:.3f}")

    # -------------------------
    # SAVE PLOTS + CSV
    # -------------------------
    forecast_plot = save_prophet_forecast_plot(dates, y_true, preds, coin)
    residual_plot = save_prophet_residual_plot(dates, y_true - preds, coin)
    pred_csv      = save_prophet_predictions_csv(dates, y_true, preds, coin)

    # -------------------------
    # APPEND RESULTS
    # -------------------------
    results_prophet.append({
        "Coin": coin,
        "Model": "Prophet",
        "Train_MAE": train_mae,
        "Train_RMSE": train_rmse,
        "Test_MAE": test_mae,
        "Test_RMSE": test_rmse,
        "ForecastPlot": str(forecast_plot),
        "ResidualPlot": str(residual_plot),
        "PredCSV": str(pred_csv)
    })

# Convert to DataFrame
print("\n================= PROPHET FINISHED =================\n")
prophet_df = pd.DataFrame(results_prophet).sort_values("Test_RMSE")
prophet_df



================= PROPHET FORECASTING START =================

----------------------------------------------------------
 Now Processing Coin: BITCOIN
----------------------------------------------------------
• Train interval: 2013-04-29 → 2019-11-15 (2392 rows)
• Test interval : 2019-11-16 → 2021-07-06 (599 rows)
 Fitting Prophet for Bitcoin...


03:17:36 - cmdstanpy - INFO - Chain [1] start processing
03:17:39 - cmdstanpy - INFO - Chain [1] done processing


 Train MAE  = 447.818
 Train RMSE = 726.966
 Test MAE   = 12141.686
 Test RMSE  = 19529.066
 Saved forecast plot: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures\Prophet_Bitcoin_forecast.png
 Saved residual plot: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures\Prophet_Bitcoin_residuals.png

----------------------------------------------------------
 Now Processing Coin: ETHEREUM
----------------------------------------------------------
• Train interval: 2015-08-08 → 2020-04-30 (1728 rows)
• Test interval : 2020-05-01 → 2021-07-06 (432 rows)
 Fitting Prophet for Ethereum...


03:17:42 - cmdstanpy - INFO - Chain [1] start processing
03:17:45 - cmdstanpy - INFO - Chain [1] done processing


 Train MAE  = 29.544
 Train RMSE = 45.830
 Test MAE   = 874.014
 Test RMSE  = 1270.163
 Saved forecast plot: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures\Prophet_Ethereum_forecast.png
 Saved residual plot: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures\Prophet_Ethereum_residuals.png

----------------------------------------------------------
 Now Processing Coin: LITECOIN
----------------------------------------------------------
• Train interval: 2013-04-29 → 2019-11-15 (2392 rows)
• Test interval : 2019-11-16 → 2021-07-06 (599 rows)
 Fitting Prophet for Litecoin...


03:17:48 - cmdstanpy - INFO - Chain [1] start processing
03:17:50 - cmdstanpy - INFO - Chain [1] done processing


 Train MAE  = 8.988
 Train RMSE = 16.333
 Test MAE   = 52.831
 Test RMSE  = 67.516
 Saved forecast plot: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures\Prophet_Litecoin_forecast.png
 Saved residual plot: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures\Prophet_Litecoin_residuals.png

================= PROPHET FINISHED =================



,Coin,Model,Train_MAE,Train_RMSE,Test_MAE,Test_RMSE,ForecastPlot,ResidualPlot,PredCSV
2,Litecoin,Prophet,8.987865,16.333301,52.830792,67.516068,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
1,Ethereum,Prophet,29.543989,45.830470,874.013720,1270.162571,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
0,Bitcoin,Prophet,447.818172,726.965811,12141.685723,19529.066362,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...


In [28]:
# ============================================================
# LSTM MODEL — ADVANCED DEEP LEARNING PIPELINE
# ============================================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path

print("\n================= LSTM FORECASTING START =================")

SEQ_LEN = 60   # 60-day input window

results_lstm = []

for coin in df["Name"].unique():

    sub = df[df["Name"] == coin][["Date", "Close"]].dropna()
    
    if len(sub) < 300:
        print(f"Skipping {coin}: Not enough data.")
        continue

    print("\n----------------------------------------------------------")
    print(f" Now Processing Coin: {coin.upper()}")
    print("----------------------------------------------------------")

    # 1) Train-test split (80-20)
    split = int(len(sub) * 0.8)
    train = sub.iloc[:split]
    test  = sub.iloc[split:]

    print(f" Train interval: {train['Date'].iloc[0].date()} → {train['Date'].iloc[-1].date()}")
    print(f" Test interval : {test['Date'].iloc[0].date()} → {test['Date'].iloc[-1].date()}")
    
    # 2) Scaling
    scaler = MinMaxScaler()
    
    train_scaled = scaler.fit_transform(train[["Close"]])
    test_scaled  = scaler.transform(test[["Close"]])

    # 3) Create sequences
    def create_sequences(data, seq_len):
        X, y = [], []
        for i in range(seq_len, len(data)):
            X.append(data[i-seq_len:i])
            y.append(data[i])
        return np.array(X), np.array(y)

    X_train, y_train = create_sequences(train_scaled, SEQ_LEN)
    X_test,  y_test  = create_sequences(test_scaled, SEQ_LEN)

    print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

    # 4) Build LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 1)),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer="adam", loss="mse")

    # 5) Train
    es = EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        callbacks=[es],
        verbose=0
    )

    # 6) Predictions
    train_pred = model.predict(X_train)
    test_pred  = model.predict(X_test)

    # invert scaling
    train_pred = scaler.inverse_transform(train_pred)
    y_train_true = scaler.inverse_transform(y_train)

    test_pred = scaler.inverse_transform(test_pred)
    y_test_true = scaler.inverse_transform(y_test)

    # 7) Compute metrics
    train_mae = mean_absolute_error(y_train_true, train_pred)
    train_rmse = root_mean_squared_error(y_train_true, train_pred)

    test_mae = mean_absolute_error(y_test_true, test_pred)
    test_rmse = root_mean_squared_error(y_test_true, test_pred)

    print(f" Train MAE  = {train_mae:.3f}")
    print(f" Train RMSE = {train_rmse:.3f}")
    print(f" Test MAE   = {test_mae:.3f}")
    print(f" Test RMSE  = {test_rmse:.3f}")

    # 8) Prepare dates
    test_dates = test["Date"].iloc[SEQ_LEN:].values

    # 9) Save forecast plot
    plt.figure(figsize=(11,5))
    plt.plot(test_dates, y_test_true.flatten(), label="Actual")
    plt.plot(test_dates, test_pred.flatten(), label="Predicted")
    plt.title(f"LSTM Forecast vs Actual — {coin}")
    plt.legend()
    plt.tight_layout()
    out_path = FIG_DIR / f"LSTM_{coin}_forecast.png"
    plt.savefig(str(out_path), dpi=240)
    plt.close()

    # 10) Save residual plot
    residuals = y_test_true.flatten() - test_pred.flatten()
    plt.figure(figsize=(11,5))
    plt.plot(test_dates, residuals)
    plt.axhline(0, color="black", linestyle="--")
    plt.title(f"LSTM Residuals — {coin}")
    plt.tight_layout()
    out_resid = FIG_DIR / f"LSTM_{coin}_residuals.png"
    plt.savefig(str(out_resid), dpi=240)
    plt.close()

    # 11) Save CSV
    predictions_df = pd.DataFrame({
        "Date": test_dates,
        "y_true": y_test_true.flatten(),
        "y_pred": test_pred.flatten()
    })
    out_csv = PROCESSED_DIR / f"predictions_lstm_{coin}.csv"
    predictions_df.to_csv(out_csv, index=False)

    # 12) Append results
    results_lstm.append({
        "Coin": coin,
        "Model": "LSTM",
        "Train_MAE": train_mae,
        "Train_RMSE": train_rmse,
        "Test_MAE": test_mae,
        "Test_RMSE": test_rmse,
        "ForecastPlot": str(out_path),
        "ResidualPlot": str(out_resid),
        "PredCSV": str(out_csv)
    })

print("\n================= LSTM FINISHED =================\n")

lstm_df = pd.DataFrame(results_lstm).sort_values("Test_RMSE")
lstm_df



================= LSTM FORECASTING START =================

----------------------------------------------------------
 Now Processing Coin: BITCOIN
----------------------------------------------------------
 Train interval: 2013-04-29 → 2019-11-15
 Test interval : 2019-11-16 → 2021-07-06
Training samples: 2332, Testing samples: 539
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
 Train MAE  = 186.994
 Train RMSE = 356.492
 Test MAE   = 2511.527
 Test RMSE  = 4303.768

----------------------------------------------------------
 Now Processing Coin: ETHEREUM
----------------------------------------------------------
 Train interval: 2015-08-08 → 2020-04-30
 Test interval : 2020-05-01 → 2021-07-06
Training samples: 1668, Testing samples: 372
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
 Train MAE  = 16.920
 Train RMSE = 28.628
 Test MAE   = 154.212
 Test RMSE  = 258.185

--------------------------------------------------

,Coin,Model,Train_MAE,Train_RMSE,Test_MAE,Test_RMSE,ForecastPlot,ResidualPlot,PredCSV
2,Litecoin,LSTM,3.351280,7.416502,7.681128,12.368029,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
1,Ethereum,LSTM,16.920257,28.627810,154.211921,258.184863,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
0,Bitcoin,LSTM,186.994460,356.492248,2511.526978,4303.768096,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
